In [1]:
import sys
import torch
import numpy as np
import pickle as pkl

sys.path.append('../../../')

from experiments.assumptions.degeneracy.script import eigenvalue_result
from models.supervised.mlp.model import MLP

models_path = "../../../models/supervised/mlp/saved_models"


In [5]:
np.random.seed(2)
torch.manual_seed(2)

In [9]:
mode = "moon"
with open(f'{models_path}/2_wide/mlp_{mode}/dataset.pkl', 'rb') as f:
	dataset = pkl.load(f)
for size in ["overfit", "2_wide", "vanilla"]:
	if size == "overfit":
		epochs = [60, 80, 199, 9999, 999, 5]
	else:
		epochs = [5, 60, 80, 199]
	for epoch in epochs:
		if size == "vanilla":
			model = MLP(2,7,10,2)
		elif size == "overfit":
			model = MLP(2,7,2,1)
		elif size == "2_wide":
			model = MLP(2,7,2,2)
		print(f"Size: {size}, Epoch: {epoch}")
		model.eval()
		model.load_state_dict(torch.load(f'{models_path}/{size}/mlp_{mode}/model_{epoch}.pth'))
		if size == "overfit":
			model.num_layers -= 1
			model.layers = model.layers[:-1]
		eigenvalue_result(dataset.X, model, N=50, labels=dataset.y, wrt="output_wise", sigma=0.05, precision=7, mode=mode, size=size, epoch=epoch)
		eigenvalue_result(dataset.X, model, N=50, labels=dataset.y, wrt="layer_wise", sigma=0.05, precision=7, mode=mode, size=size, epoch=epoch)

Size: overfit, Epoch: 60
Size: overfit, Epoch: 80
Size: overfit, Epoch: 199
Size: overfit, Epoch: 9999
Size: overfit, Epoch: 999
Size: overfit, Epoch: 5
Size: 2_wide, Epoch: 5
Size: 2_wide, Epoch: 60
Size: 2_wide, Epoch: 80
Size: 2_wide, Epoch: 199
Size: vanilla, Epoch: 5


/Users/maxpowers/miniconda3/envs/DL/lib/python3.10/site-packages/matplotlib/cbook/__init__.py:1335: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


Size: vanilla, Epoch: 60
Size: vanilla, Epoch: 80
Size: vanilla, Epoch: 199


In [4]:
mode = "blobs"
with open(f'{models_path}/vanill/mlp_{mode}/dataset.pkl', 'rb') as f:
	dataset = pkl.load(f)
epochs = [5, 60, 80, 199]

for size in ["vanilla"]:
	for epoch in epochs:
		if size == "vanilla":
			model = MLP(2,7,4,4)
		print(f"Size: {size}, Epoch: {epoch}")
		model.eval()
		model.load_state_dict(torch.load(f'{models_path}/{size}/mlp_{mode}/model_{epoch}.pth'))
		eigenvalue_result(dataset.X, model, N=50, labels=dataset.y, wrt="output_wise", sigma=0.05, precision=7, mode=mode, size=size, epoch=epoch)
		eigenvalue_result(dataset.X, model, N=50, labels=dataset.y, wrt="layer_wise", sigma=0.05, precision=7, mode=mode, size=size, epoch=epoch)

FileNotFoundError: [Errno 2] No such file or directory: '../../../models/supervised/mlp/saved_models/vanill/mlp_blobs/dataset.pkl'

In [ ]:

def matrix_signature(A):
    # Compute eigenvalues
    eigenvalues = np.linalg.eigvals(A)
    
    n_pos = np.sum(eigenvalues > 1e-10, axis=1)  # Number of positive eigenvalues
    n_neg = np.sum(eigenvalues < -1e-10, axis=1) # Number of negative eigenvalues
    n_zero = np.sum(np.abs(eigenvalues) <= 1e-10, axis=1) # Number of zero eigenvalues
    
    return (n_pos, n_neg, n_zero)

signature = matrix_signature(g_)